In [1]:
import rasterio

tif_path =r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack.tif"

with rasterio.open(tif_path) as src:
    print("NoData value:", src.nodata)

NoData value: nan


In [2]:
import rasterio
import numpy as np

tif_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack.tif"

with rasterio.open(tif_path) as src:
    data = src.read(masked=True)  # masked=True replaces NoData with masked entries
    total_pixels = data.size
    missing_pixels = np.ma.count_masked(data)
    missing_percentage = (missing_pixels / total_pixels) * 100

print(f"Total pixels: {total_pixels:,}")
print(f"Missing pixels: {missing_pixels:,}")
print(f"Missing percentage: {missing_percentage:.2f}%")


Total pixels: 809,710,080
Missing pixels: 23,487,325
Missing percentage: 2.90%


In [ ]:
from rasterio.fill import fillnodata
import rasterio
import numpy as np
from pathlib import Path

in_tif = Path(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\ERA5_30m_stack.tif")
out_tif = in_tif.with_name(in_tif.stem + "_filled.tif")

with rasterio.open(in_tif) as src:
    arr = src.read(1).astype('float32')
    nod = src.nodatavals[0]
    if nod is not None:
        arr[arr == nod] = np.nan
    mask = np.isnan(arr)

    filled = fillnodata(arr, mask=mask, max_search_distance=100, smoothing_iterations=0)

    profile = src.profile.copy()
    profile.update(dtype=rasterio.float32, nodata=None, compress='LZW')

    with rasterio.open(out_tif, 'w', **profile) as dst:
        dst.write(filled, 1)

print("Saved filled ERA5:", out_tif)